In [ ]:
# Installing libraries
!pip install pathway bokeh

In [ ]:
# Importing libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from datetime import datetime
import pathway as pw
import bokeh.plotting
import panel as pn

In [ ]:
# Loading data
data = pd.read_csv('/content/dataset.csv')
data

In [ ]:
data.info()

In [ ]:
# Sorting according to timestamp
data['Timestamp'] = pd.to_datetime(data['LastUpdatedDate'] + ' ' + data['LastUpdatedTime'],
                                  format='%d-%m-%Y %H:%M:%S')
data = data.sort_values('Timestamp').reset_index(drop=True)
data.head()

In [ ]:
# Filtering data for baseline linear model
data[["SystemCodeNumber", "Timestamp", "Occupancy", "Capacity"]].to_csv("baseline_linear_parking_stream.csv", index=False)

In [ ]:
# Defining schema for baseline linear model
class Baseline_Linear_ParkingSchema(pw.Schema):
    SystemCodeNumber: str
    Timestamp: str
    Occupancy: int
    Capacity: int

In [ ]:
# Simulating real time data stream
baseline_linear_data_parking = pw.demo.replay_csv("baseline_linear_parking_stream.csv", schema = Baseline_Linear_ParkingSchema, input_rate=1000)

In [ ]:
fmt = "%Y-%m-%d %H:%M:%S"
data_with_time = baseline_linear_data_parking.with_columns(
    t = baseline_linear_data_parking.Timestamp.dt.strptime(fmt),
    day = baseline_linear_data_parking.Timestamp.dt.strptime(fmt).dt.strftime("%Y-%m-%dT00:00:00"))

In [ ]:
data_utilization = data_with_time.with_columns(
    utilization=pw.this.Occupancy / pw.this.Capacity
)

In [ ]:
# Baseline Linear Model
def update_price():
    state = {}
    def price_func(system_code, t, utilization, occ, cap):
        alpha = 2.0
        previous_price = state.get(system_code, 10.0)
        new_price = previous_price + alpha * utilization
        state[system_code] = new_price
        return new_price

    return price_func

price_stream = data_utilization.with_columns(
    price=update_price()(
        pw.this.SystemCodeNumber,
        pw.this.t,
        pw.this.utilization,
        pw.this.Occupancy,
        pw.this.Capacity
    )
)

In [ ]:
# Plot Baseline Linear Model
from bokeh.models import ColumnDataSource
pn.extension()

parking_lots = ['BHMBCCMKT01', 'BHMNCPHST01', 'BHMMBMMBX01', 'BHMNCPNST01',
       'Shopping', 'BHMEURBRD01', 'Broad Street', 'Others-CCCPS8',
       'Others-CCCPS105a', 'Others-CCCPS119a', 'BHMBCCTHL01',
       'Others-CCCPS135a', 'Others-CCCPS202', 'Others-CCCPS98']

def lot_price_plot(system_code):
    def plot_func(source):
        fig = bokeh.plotting.figure(
            height=300,
            width=500,
            title=f"Price Dynamics: {system_code}",
            x_axis_type="datetime",
            y_range = (10, 20)
        )
        fig.line("t", "price", source=source, line_width=2, color="navy")
        fig.scatter("t", "price", source=source, size=5, color="red")
        return fig
    return plot_func

viz_dict = {
    lot: price_stream.filter(pw.this.SystemCodeNumber == lot).plot(
        lot_price_plot(lot), sorting_col="t"
    )
    for lot in parking_lots
}

column1 = [viz_dict[lot] for lot in parking_lots[:7]]
column2 = [viz_dict[lot] for lot in parking_lots[7:]]

dashboard = pn.Row(
    pn.Column(*column1),
    pn.Column(*column2)
)

dashboard.servable()


In [ ]:
# Run Baseline Linear Model
%%capture --no-display
pw.run()